# Network Analysis for BeveL Betaseries 

Inputs: betaseries files for BeveL participants (n=85) drawn from 4 conditions: choice, reward taste, punishment taste, neutral rinse

Analysis workflow is mapped off this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5429248/


### Input Data
- One timeseries txt file per participant. Conditions must be separated in unique folder
- Need a txt file of the labels for each ROI

### Outputs
- Circle graph showing thresholded connectivity of each ROI. ROI color denotes modules. 
- Circle graph of module edge weights. 
- CSV file with module assignment & nodal metrics for each ROI

### Running the notebook
The following parts of the code should be changed
1. Filepath to timeseries 
2. Name of circle graph figure
3. Module dict to assign to module graph
4. Name of module graph figure
5. Name of csv file

## Note: 
### Print Statements are commented out to save time, remove comments if desired. 

In [1]:
# Imports 
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import bz2
import pickle
import community
import statistics
import pdb
from scipy import stats
import scipy
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj, ColorbarObj
from visbrain.io import download_file


/anaconda3/lib/python3.6/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


In [2]:
basepath = '/Users/jennygilbert/Documents/'

## Load in the data

### Find the path to the data

In [3]:
#Find the path to data
file_list = glob.glob(os.path.join(basepath,'betaseries_bevel/4_combine_timeseries/reward/*.txt'))

In [4]:
#Check the files found
print(file_list)

['/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-001_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-002_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-003_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-004_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-005_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-006_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-007_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-009_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-010_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-011_reward.txt', '/Users/jennygilbert/Document

In [5]:
#Check to see how many participants 
#len(file_list)

### Make a dictionary to read in the files to pandas

In [6]:
#Setting the ditionary
my_dict={}
for item in file_list:
    name=item.split('/')[7].split('.')[0]
    print(name)
    my_dict.setdefault(name, []).append(item)

sub-001_reward
sub-002_reward
sub-003_reward
sub-004_reward
sub-005_reward
sub-006_reward
sub-007_reward
sub-009_reward
sub-010_reward
sub-011_reward
sub-012_reward
sub-013_reward
sub-014_reward
sub-015_reward
sub-016_reward
sub-017_reward
sub-018_reward
sub-019_reward
sub-020_reward
sub-021_reward
sub-022_reward
sub-024_reward
sub-025_reward
sub-026_reward
sub-027_reward
sub-028_reward
sub-029_reward
sub-030_reward
sub-031_reward
sub-032_reward
sub-033_reward
sub-034_reward
sub-035_reward
sub-036_reward
sub-037_reward
sub-038_reward
sub-039_reward
sub-040_reward
sub-041_reward
sub-042_reward
sub-043_reward
sub-044_reward
sub-045_reward
sub-046_reward
sub-047_reward
sub-048_reward
sub-050_reward
sub-052_reward
sub-053_reward
sub-054_reward
sub-055_reward
sub-056_reward
sub-057_reward
sub-058_reward
sub-059_reward
sub-060_reward
sub-061_reward
sub-062_reward
sub-063_reward
sub-064_reward
sub-066_reward
sub-067_reward
sub-068_reward
sub-069_reward
sub-070_reward
sub-071_reward
sub-072_re

In [7]:
# Checking to make sure its populated
#my_dict['sub-024_reward']

### Read the data from the dictionary into pandas

In [8]:
#Setting the data dictionary
data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))

In [9]:
#Check the dictionary
data_dict.keys()

dict_keys(['sub-001_reward', 'sub-002_reward', 'sub-003_reward', 'sub-004_reward', 'sub-005_reward', 'sub-006_reward', 'sub-007_reward', 'sub-009_reward', 'sub-010_reward', 'sub-011_reward', 'sub-012_reward', 'sub-013_reward', 'sub-014_reward', 'sub-015_reward', 'sub-016_reward', 'sub-017_reward', 'sub-018_reward', 'sub-019_reward', 'sub-020_reward', 'sub-021_reward', 'sub-022_reward', 'sub-024_reward', 'sub-025_reward', 'sub-026_reward', 'sub-027_reward', 'sub-028_reward', 'sub-029_reward', 'sub-030_reward', 'sub-031_reward', 'sub-032_reward', 'sub-033_reward', 'sub-034_reward', 'sub-035_reward', 'sub-036_reward', 'sub-037_reward', 'sub-038_reward', 'sub-039_reward', 'sub-040_reward', 'sub-041_reward', 'sub-042_reward', 'sub-043_reward', 'sub-044_reward', 'sub-045_reward', 'sub-046_reward', 'sub-047_reward', 'sub-048_reward', 'sub-050_reward', 'sub-052_reward', 'sub-053_reward', 'sub-054_reward', 'sub-055_reward', 'sub-056_reward', 'sub-057_reward', 'sub-058_reward', 'sub-059_reward',

In [10]:
#Check for the dataframe
#data_dict['sub-058_reward']

### Create a new dictionary with correlation matrix

In [11]:
#Setting up the correlation dictionary
cor_dict={}

for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    cor_dict[key] = cor_matrix
    

In [12]:
#check the dictionary
#list(cor_dict.values())[3]

### Make a dictionary of labels for the nodes

In [13]:
#This points to a txt file with the ROI names in a list separated by commas
path = os.path.join(basepath,'betaseries_bevel/5_analysis/labels.txt')
df_label = pd.read_csv(path, sep=',')

#df_label.head()

In [14]:
labels_dict = {}
n=0
for item in df_label:
    labels_dict[n]=item
    n=n+1

In [15]:
print(labels_dict)

{0: 'Amygdala_L', 1: 'Amygdala_R', 2: 'Dorsal_striatum_L', 3: 'Dorsal_striatum_R', 4: 'Fusiform_gyrus_L', 5: 'Fusiform_gyrus_R', 6: 'Hippocampus_L', 7: 'Hippocampus_R', 8: 'Insula_L', 9: 'Insula_R', 10: 'Intracalcarine_cortex_L', 11: 'Intracalcarine_cortex_R', 12: 'lOFC_L', 13: 'lOFC_R', 14: 'mOFC_L', 15: 'mOFC_R', 16: 'Oral_somatosensory_cortex_R', 17: 'Oral_somatosensory_cortex_L', 18: 'Precuneus_L', 19: 'Precuneus_R', 20: 'Ventral_striatum_L', 21: 'Ventral_striatum_R', 22: 'vlPFC_L', 23: 'vlPFC_R', 24: 'vlThalamus_L', 25: 'vlThalamus_R', 26: 'vmPFC_L', 27: 'vmPFC_R'}


### Function to create a graph with positive or negative values and minimum correlation value

In [16]:
def create_corr_network_5(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

### Function to make a graph object BY SUBJECT
This will return:
- The edges (noramlized R correlation matrix, in pandas dataframe)
- The correlations (absolute value of the edges in a numpy dataframe)
- The mean_FC (the mean functional connectivity per subject/run)
- The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)
- The modules (communitites in the network at the participant level

## Changes from the First Iteration

1. Test for mean,sd, and module assignment distribution among participant's graphs

In [17]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    cor_dict={}
    FC_dict={}
    sd_dict={}
    graph_dict={}
    partition_dict={}
    for key, values in list_o_data.items():
            #i=i.set_index(labels.ID)
            #i.rename(columns=labels.ID, inplace=True)
            ########################################
            edge_dict.setdefault(key, []).append(values)
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            sd=x.std()
            ########################################
            cor_dict.setdefault(key, []).append(x)
            ########################################
            FC_dict.setdefault(key, []).append(mu)
            sd_dict.setdefault(key, []).append(sd)
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            #for i, nlrow in labels.iterrows():
                #G.node[i].update(nlrow[0:].to_dict())
            ########################################
            graph_dict.setdefault(key, []).append(G)
            ########################################
            partition = community.best_partition(create_corr_network_5(G, direction,min_cor))
            ########################################
            partition_dict.setdefault(key, []).append(partition)
            ########################################
    return({'edges':edge_dict, 'correlations':cor_dict, 'mean_FC':FC_dict, 'stdev':sd_dict, 'graphs':graph_dict,'modules':partition_dict})

### Apply the function to correlations & check output

In [18]:
# Apply function
GRAPHS = make_graphs(cor_dict, "positive", 0)

In [19]:
# Check the keys for the dictionary
GRAPHS.keys()

dict_keys(['edges', 'correlations', 'mean_FC', 'stdev', 'graphs', 'modules'])

In [23]:
# Check modules for one subject
#GRAPHS['stdev']['sub-001_punish']

In [24]:
#Check to make sure graphs are filled
#Test = GRAPHS['graphs']['sub-001_reward'][0]
#Test.edges(data=True)

In [25]:
GRAPHS['mean_FC']

{'sub-001_reward': [0.2848602794212194],
 'sub-002_reward': [0.35856202261993647],
 'sub-003_reward': [0.23622929390545463],
 'sub-004_reward': [0.21269017088271103],
 'sub-005_reward': [0.37057148035938847],
 'sub-006_reward': [0.2263212903943031],
 'sub-007_reward': [0.2534808597784768],
 'sub-009_reward': [0.2965987229797016],
 'sub-010_reward': [0.286494156389793],
 'sub-011_reward': [0.49835533845313185],
 'sub-012_reward': [0.3585074814383908],
 'sub-013_reward': [0.3183437385688916],
 'sub-014_reward': [0.24565786120820746],
 'sub-015_reward': [0.29886493927261676],
 'sub-016_reward': [0.3111134721329929],
 'sub-017_reward': [0.27933232701441335],
 'sub-018_reward': [0.26954354391005303],
 'sub-019_reward': [0.2605883787686915],
 'sub-020_reward': [0.2509311019470671],
 'sub-021_reward': [0.21474761152305286],
 'sub-022_reward': [0.2552437366975302],
 'sub-024_reward': [0.2531467778227901],
 'sub-025_reward': [0.2682607227565468],
 'sub-026_reward': [0.26854081462923146],
 'sub-

### Get the standard deviation of the mean correlation

In [26]:
#statistics.stdev(GRAPHS['mean_FC'])
m=[]

for key,value in GRAPHS['mean_FC'].items():
    m.append(value[0])

In [27]:
#describe the mean correlation of participant's graphs
stats.describe(m)

DescribeResult(nobs=85, minmax=(0.20383900223774987, 0.5166211767218827), mean=0.30456663895510727, variance=0.00413335833745449, skewness=1.1011435443430952, kurtosis=1.4490091512477061)

In [28]:
statistics.mean(m)

0.30456663895510727

In [29]:
m

[0.2848602794212194,
 0.35856202261993647,
 0.23622929390545463,
 0.21269017088271103,
 0.37057148035938847,
 0.2263212903943031,
 0.2534808597784768,
 0.2965987229797016,
 0.286494156389793,
 0.49835533845313185,
 0.3585074814383908,
 0.3183437385688916,
 0.24565786120820746,
 0.29886493927261676,
 0.3111134721329929,
 0.27933232701441335,
 0.26954354391005303,
 0.2605883787686915,
 0.2509311019470671,
 0.21474761152305286,
 0.2552437366975302,
 0.2531467778227901,
 0.2682607227565468,
 0.26854081462923146,
 0.4087092701281954,
 0.2064481890269867,
 0.25536399057669185,
 0.30401885920967797,
 0.34980383366628465,
 0.23256744034878715,
 0.27804698772816977,
 0.33676209376527194,
 0.2511652398528155,
 0.2853068206610137,
 0.29697142305770025,
 0.3776642351109596,
 0.21917925287754073,
 0.2637785299819304,
 0.2300062603870038,
 0.3113557640743733,
 0.34631155512462036,
 0.4512244868545139,
 0.4037509712670373,
 0.3751469738137319,
 0.27888314709246814,
 0.22649027734012997,
 0.2873331183

In [30]:
#Show the histogram of average correlation across participant's graphs

hist, bin_edges = scipy.histogram(m, bins = 10) 
  
# Checking the results 
print ("No. of points in each bin : ", hist) 
print ("Size of the bins          : ", bin_edges) 
  
# plotting the histogram 
plt.bar(bin_edges[:-1], hist, width = .1) 
plt.xlim(min(bin_edges), max(bin_edges)) 
#plt.show() 

No. of points in each bin :  [10 12 23 17  9  7  3  1  0  3]
Size of the bins          :  [0.203839   0.23511722 0.26639544 0.29767365 0.32895187 0.36023009
 0.39150831 0.42278652 0.45406474 0.48534296 0.51662118]


(0.20383900223774987, 0.5166211767218827)

### Get the mean SD of participant's corrleation matricies

In [31]:
sd=[]
for key,value in GRAPHS['stdev'].items():
    sd.append(value[0])

In [32]:
#describe the mean correlation of participant's graphs
stats.describe(sd)

DescribeResult(nobs=85, minmax=(0.1936001322707724, 0.3146723695461026), mean=0.22438820713991522, variance=0.0005766317185922301, skewness=1.7884612319790045, kurtosis=3.3764391902152617)

In [33]:
sd

[0.2073635207034673,
 0.2733026006923,
 0.2116172710089582,
 0.20231125522696475,
 0.2410264565999763,
 0.2120057561819862,
 0.2000880855674745,
 0.21406337214449028,
 0.20149595349917296,
 0.309505192889523,
 0.2146746901387391,
 0.23344361962899463,
 0.22307271138356988,
 0.2273662366620012,
 0.22307133204233248,
 0.20093487440776553,
 0.20607379509819654,
 0.21540642700392532,
 0.21265325875708518,
 0.2079075807873622,
 0.21230339323069403,
 0.21517966564281107,
 0.20941222038440507,
 0.22469549026727342,
 0.27727457736067507,
 0.19784963281008813,
 0.21037875419566252,
 0.21849367670830655,
 0.23960716483777042,
 0.1994663401962697,
 0.20923784251926192,
 0.21361819146886052,
 0.20558676612867494,
 0.21596321561875687,
 0.21315010665209583,
 0.26311495624365044,
 0.20759117685554077,
 0.21720824572372158,
 0.20608522182753589,
 0.20151171472685883,
 0.21219636837457945,
 0.2134112529738157,
 0.24373894038860436,
 0.26613047822167124,
 0.2275896737124343,
 0.204328522747463,
 0.2175

In [34]:
hist, bin_edges = scipy.histogram(sd, bins = 10) 
  
# Checking the results 
print ("No. of points in each bin : ", hist) 
print ("Size of the bins          : ", bin_edges) 
  
# plotting the histogram 
plt.bar(bin_edges[:-1], hist, width = .05) 
plt.xlim(min(bin_edges), max(bin_edges)) 
#plt.show() 

No. of points in each bin :  [14 31 17 11  3  3  2  2  0  2]
Size of the bins          :  [0.19360013 0.20570736 0.21781458 0.2299218  0.24202903 0.25413625
 0.26624347 0.2783507  0.29045792 0.30256515 0.31467237]


(0.1936001322707724, 0.3146723695461026)

If this value is low (<0.1) then you don't need to threshold the graph? 

### Test Modularity

Goal: use modularity function from communities to identify module structure that emerge during reward within the sample

In [35]:
#this is pulling out the module structure for each participant
modules=[]

for key,value in GRAPHS['modules'].items():
    modules.append(value[0])

In [36]:
#print it to make sure it works
modules

[{0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 4,
  6: 0,
  7: 0,
  8: 1,
  9: 1,
  10: 4,
  11: 4,
  12: 4,
  13: 1,
  14: 4,
  15: 4,
  16: 2,
  17: 2,
  18: 1,
  19: 1,
  20: 5,
  21: 5,
  22: 1,
  23: 2,
  24: 1,
  25: 1,
  26: 5,
  27: 2},
 {0: 0,
  1: 0,
  2: 1,
  3: 1,
  4: 2,
  5: 2,
  6: 0,
  7: 0,
  8: 1,
  9: 1,
  10: 2,
  11: 2,
  12: 3,
  13: 3,
  14: 2,
  15: 2,
  16: 1,
  17: 1,
  18: 4,
  19: 4,
  20: 4,
  21: 4,
  22: 1,
  23: 1,
  24: 0,
  25: 0,
  26: 1,
  27: 1},
 {0: 0,
  1: 0,
  2: 1,
  3: 1,
  4: 2,
  5: 2,
  6: 0,
  7: 0,
  8: 3,
  9: 3,
  10: 3,
  11: 3,
  12: 1,
  13: 3,
  14: 2,
  15: 2,
  16: 1,
  17: 1,
  18: 3,
  19: 3,
  20: 0,
  21: 0,
  22: 1,
  23: 4,
  24: 5,
  25: 5,
  26: 2,
  27: 1},
 {0: 0,
  1: 0,
  2: 1,
  3: 1,
  4: 2,
  5: 3,
  6: 0,
  7: 0,
  8: 4,
  9: 4,
  10: 3,
  11: 3,
  12: 4,
  13: 0,
  14: 2,
  15: 2,
  16: 1,
  17: 1,
  18: 0,
  19: 5,
  20: 1,
  21: 1,
  22: 6,
  23: 6,
  24: 4,
  25: 4,
  26: 1,
  27: 1},
 {0: 0,
  1: 0,
  2: 1,
  3:

In [37]:
#made a df of the modules
df = pd.DataFrame.from_dict(modules)

In [38]:
df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0,1,2,3,4,4,0,0,1,1,...,1,1,5,5,1,2,1,1,5,2
1,0,0,1,1,2,2,0,0,1,1,...,4,4,4,4,1,1,0,0,1,1
2,0,0,1,1,2,2,0,0,3,3,...,3,3,0,0,1,4,5,5,2,1
3,0,0,1,1,2,3,0,0,4,4,...,0,5,1,1,6,6,4,4,1,1
4,0,0,1,1,2,2,0,0,1,3,...,2,4,4,4,1,1,2,2,1,1
5,0,1,2,2,3,4,3,3,0,0,...,4,4,2,2,1,1,0,0,3,3
6,0,0,1,1,2,3,0,0,4,4,...,3,1,0,0,5,5,0,0,5,5
7,0,0,1,2,3,3,0,0,3,3,...,5,5,2,2,2,2,3,3,2,2
8,0,1,2,2,1,0,0,1,3,4,...,0,4,6,4,6,6,3,3,6,4
9,0,0,1,1,0,1,0,0,1,1,...,0,1,2,0,0,0,0,2,0,1


In [39]:
#label the modules
df.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_R", 17:"Oral_somatosensory_cortex_L", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R
0,0,1,2,3,4,4,0,0,1,1,...,1,1,5,5,1,2,1,1,5,2
1,0,0,1,1,2,2,0,0,1,1,...,4,4,4,4,1,1,0,0,1,1
2,0,0,1,1,2,2,0,0,3,3,...,3,3,0,0,1,4,5,5,2,1
3,0,0,1,1,2,3,0,0,4,4,...,0,5,1,1,6,6,4,4,1,1
4,0,0,1,1,2,2,0,0,1,3,...,2,4,4,4,1,1,2,2,1,1
5,0,1,2,2,3,4,3,3,0,0,...,4,4,2,2,1,1,0,0,3,3
6,0,0,1,1,2,3,0,0,4,4,...,3,1,0,0,5,5,0,0,5,5
7,0,0,1,2,3,3,0,0,3,3,...,5,5,2,2,2,2,3,3,2,2
8,0,1,2,2,1,0,0,1,3,4,...,0,4,6,4,6,6,3,3,6,4
9,0,0,1,1,0,1,0,0,1,1,...,0,1,2,0,0,0,0,2,0,1


# Plot the distibution of module assignments for the ROIs

In [40]:
#df.hist()

### Only a couple ROI have stable module assignment -- modules 0, 1, and 2 
- 0:"Amygdala_L"
- 1:"Amygdala_R"
- 2:"Dorsal_striatum_L"

In [41]:
#find the mean over the columns
df.mean(axis = 0)

0     0.000000
1     0.352941
2     1.247059
3     1.341176
4     1.623529
5     2.129412
6     1.188235
7     1.000000
8     2.305882
9     2.388235
10    2.611765
11    2.623529
12    2.647059
13    2.776471
14    2.223529
15    2.494118
16    1.811765
17    1.764706
18    2.270588
19    2.364706
20    2.305882
21    2.164706
22    2.564706
23    2.635294
24    2.611765
25    2.482353
26    2.670588
27    2.670588
dtype: float64

In [42]:
# Check the standard deviation over the columns
df.std(axis = 0)

0     0.000000
1     0.480721
2     0.554331
3     0.732786
4     1.164724
5     1.251554
6     1.483995
7     1.422607
8     1.488236
9     1.472721
10    1.215908
11    1.184990
12    1.493684
13    1.483051
14    1.514819
15    1.570741
16    1.304700
17    1.259696
18    1.892266
19    1.724108
20    1.845351
21    1.901422
22    1.886336
23    1.765051
24    1.780143
25    1.783601
26    1.808632
27    1.720937
dtype: float64



## The standard deviation of module assignments is high (>1) for the majority of the modules. 

### This suggests the module assignment is not stable across participants 

### Using the median module assignment across participants makes more sense than the mean

In [43]:
#find the mean over the columns
partition_median = df.median()
df_median = pd.DataFrame(partition_median)

In [44]:
# View the median module for each node
df_median

,0
0,0.0
1,0.0
2,1.0
3,1.0
4,2.0
5,2.0
6,0.0
7,0.0
8,2.0
9,3.0


### Make a Dicitionary with the modules

In [45]:
#Convert modules to dictionary
#mod_dict={}
#for index, row in df_median.iterrows():
    #mod_dict[index]= row[0]

#mod_dict

### Combine participant correlation matrices into one mean OR MEDIAN correlation matrix

In [46]:
#make the function to combine
def make_total_graphs(dict_o_data):
    mylist=[]
    for key, val_list in dict_o_data.items():
        for i in val_list:
            cor_matrix = np.asarray(i)
            mylist.append(cor_matrix)
    x=np.stack(mylist, axis=2)
    mu=np.median(x, axis=(2))
    #mu=np.mean(x, axis=(2))
    return(mu)

In [47]:
#Make the mean graph with correlations
mean_graph = make_total_graphs(GRAPHS['correlations'])

In [48]:
#Check to make sure this worked 
print(mean_graph.shape)

#Convert the graph to a numpy matrix so it can be recognized by networkX
mean_graph_mat = np.matrix(mean_graph)

#Check the mean correlation to use to threshold later
mean_graph_mat.mean()

(28, 28)


0.2832574556194909

In [49]:
meanG = nx.from_numpy_matrix(mean_graph_mat)

# Check Modularity in the Mean Graph

### This is the way Cohen's group usually calculates modularity, because its more stable

In [50]:
partition = community.best_partition(meanG)

In [51]:
#Add the modules and ROI labels to the graph
nx.set_node_attributes(meanG, partition, 'modules')
nx.set_node_attributes(meanG, labels_dict, 'ROIs')

In [52]:
# Check to make sure this worked
ROIs=nx.get_node_attributes(meanG,'ROIs')
ROIs

{0: 'Amygdala_L',
 1: 'Amygdala_R',
 2: 'Dorsal_striatum_L',
 3: 'Dorsal_striatum_R',
 4: 'Fusiform_gyrus_L',
 5: 'Fusiform_gyrus_R',
 6: 'Hippocampus_L',
 7: 'Hippocampus_R',
 8: 'Insula_L',
 9: 'Insula_R',
 10: 'Intracalcarine_cortex_L',
 11: 'Intracalcarine_cortex_R',
 12: 'lOFC_L',
 13: 'lOFC_R',
 14: 'mOFC_L',
 15: 'mOFC_R',
 16: 'Oral_somatosensory_cortex_R',
 17: 'Oral_somatosensory_cortex_L',
 18: 'Precuneus_L',
 19: 'Precuneus_R',
 20: 'Ventral_striatum_L',
 21: 'Ventral_striatum_R',
 22: 'vlPFC_L',
 23: 'vlPFC_R',
 24: 'vlThalamus_L',
 25: 'vlThalamus_R',
 26: 'vmPFC_L',
 27: 'vmPFC_R'}

In [53]:
#Check if this worked
partitions=nx.get_node_attributes(meanG,'modules')
partitions

{0: 0,
 1: 0,
 2: 1,
 3: 1,
 4: 2,
 5: 3,
 6: 0,
 7: 0,
 8: 4,
 9: 4,
 10: 3,
 11: 3,
 12: 3,
 13: 3,
 14: 2,
 15: 2,
 16: 1,
 17: 1,
 18: 5,
 19: 5,
 20: 6,
 21: 6,
 22: 1,
 23: 1,
 24: 4,
 25: 4,
 26: 1,
 27: 1}

In [107]:
d1 = dict(list(GRAPHS.items())[len(GRAPHS)//2:])
d2 = dict(list(GRAPHS.items())[:len(GRAPHS)//2])

In [112]:
d2.keys()

dict_keys(['edges', 'correlations', 'mean_FC'])

# Module Connectivity

In [54]:
#Apply the partition to participant's data
modg = community.induced_graph(partitions, meanG) 

### Function to make a thresholded graph

In [55]:
def threshold(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

In [56]:
threshG = threshold(meanG, 'positive', 0.3)

In [57]:
meanG.nodes(data=True)

NodeDataView({0: {'modules': 0, 'ROIs': 'Amygdala_L'}, 1: {'modules': 0, 'ROIs': 'Amygdala_R'}, 2: {'modules': 1, 'ROIs': 'Dorsal_striatum_L'}, 3: {'modules': 1, 'ROIs': 'Dorsal_striatum_R'}, 4: {'modules': 2, 'ROIs': 'Fusiform_gyrus_L'}, 5: {'modules': 3, 'ROIs': 'Fusiform_gyrus_R'}, 6: {'modules': 0, 'ROIs': 'Hippocampus_L'}, 7: {'modules': 0, 'ROIs': 'Hippocampus_R'}, 8: {'modules': 4, 'ROIs': 'Insula_L'}, 9: {'modules': 4, 'ROIs': 'Insula_R'}, 10: {'modules': 3, 'ROIs': 'Intracalcarine_cortex_L'}, 11: {'modules': 3, 'ROIs': 'Intracalcarine_cortex_R'}, 12: {'modules': 3, 'ROIs': 'lOFC_L'}, 13: {'modules': 3, 'ROIs': 'lOFC_R'}, 14: {'modules': 2, 'ROIs': 'mOFC_L'}, 15: {'modules': 2, 'ROIs': 'mOFC_R'}, 16: {'modules': 1, 'ROIs': 'Oral_somatosensory_cortex_R'}, 17: {'modules': 1, 'ROIs': 'Oral_somatosensory_cortex_L'}, 18: {'modules': 5, 'ROIs': 'Precuneus_L'}, 19: {'modules': 5, 'ROIs': 'Precuneus_R'}, 20: {'modules': 6, 'ROIs': 'Ventral_striatum_L'}, 21: {'modules': 6, 'ROIs': 'Vent

### Function to visualize thresholded graph with modules in colors

In [58]:
def jenny_graph(graph):
    edges,weights = zip(*nx.get_edge_attributes(graph, 'weight').items())
    nodes, color = zip(*nx.get_node_attributes(graph,'modules').items()) #if your modules are named different change here
    nodes, positions = zip(*nx.get_node_attributes(graph,'ROIs').items())
    #positions
    positions=nx.circular_layout(graph) #this is defining a circluar graph, if you want a different one you change the circular part of this line
    
    #Figure size
    plt.figure(figsize=(40,25))
    
    
    #draws nodes
    color = np.array(color)
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    y=nx.draw_networkx_nodes(graph,positions, 
                           node_color=color,
                           node_size=4000,
                           alpha=0.8, 
                           cmap= nColormap,
                           vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(graph, positions, labels = ROIs, font_size=10, 
                            font_family='sans-serif', fontweight = 'bold')
    
    
    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.bwr #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    wt=weights*5
    M=wt.max()
    m=wt.min()
    x=nx.draw_networkx_edges(graph, positions, edge_list=edges, style='solid', width = wt, edge_color = wt,
                           cmap=eColormap,
                           edge_vmin=m,
                           edge_vmax=M)
    
    #format the colorbar
    node_bar=plt.colorbar(y)
    edge_bar=plt.colorbar(x)

    node_bar.set_label('Modularity',fontsize = 25)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)

    plt.axis('off')
    plt.title("Modularity and Edge Weights of Average Graph", fontsize = 30)
    #plt.savefig(os.path.join(basepath,"betaseries_bevel/5_analysis/modularity_circle_reward.png", format="PNG")
    plt.show()

In [59]:
jenny_graph(threshG)

# USE BCTPY FOR PARTICIPATION COEFFICIENT

In [60]:
import bct

In [61]:
#Participation coefficient usage
#bct.participation_coef(W, ci, degree='undirected')¶

In [62]:
b = np.array(list(partition.keys()))

In [63]:
cor_mat = np.array(mean_graph_mat)
test_array=np.array(list(list(b)))
testPART=np.vstack(test_array)
PC=bct.participation_coef(W=cor_mat, ci= testPART)

In [64]:
PC

array([0.94759905, 0.94822527, 0.95397782, 0.95517909, 0.95109594,
       0.94827182, 0.95420413, 0.95077318, 0.95619307, 0.95442523,
       0.95012778, 0.95114731, 0.95191112, 0.95081392, 0.95055646,
       0.95245103, 0.95346107, 0.95469738, 0.94337874, 0.95306133,
       0.95405003, 0.95136988, 0.95076865, 0.94961188, 0.95271834,
       0.95417371, 0.95369551, 0.95384468])

### Make Module Graph Connectivity Weights

Make a new graph with modules from whole group analysis 

In [65]:
modg = community.induced_graph(partitions, meanG)

In [66]:
nx.edges(modg, nbunch=None)

EdgeView([(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 3), (3, 4), (3, 5), (3, 6), (4, 4), (4, 5), (4, 6), (5, 5), (5, 6), (6, 6)])

In [67]:
nx.get_edge_attributes(modg, "weight")

{(0, 0): 6.460822928736948,
 (0, 1): 6.233851134415081,
 (0, 2): 2.7687511531004567,
 (0, 3): 3.663988157410524,
 (0, 4): 3.785557688261466,
 (0, 5): 1.5690876225415686,
 (0, 6): 2.219965065140905,
 (1, 1): 20.496692191209103,
 (1, 2): 5.220893966307204,
 (1, 3): 8.440344290282296,
 (1, 4): 9.690938501445075,
 (1, 5): 3.8730220869705185,
 (1, 6): 4.4974434731146475,
 (2, 2): 4.24194900192367,
 (2, 3): 4.032391856488529,
 (2, 4): 2.6235133759700573,
 (2, 5): 0.995404889054534,
 (2, 6): 1.152438633006798,
 (3, 3): 8.798469432775416,
 (3, 4): 4.999651699306607,
 (3, 5): 1.747275788273452,
 (3, 6): 1.8417465402521869,
 (4, 4): 6.610128528706005,
 (4, 5): 1.5722161332293596,
 (4, 6): 1.8958949301647774,
 (5, 5): 2.337923529567626,
 (5, 6): 0.8322065508888203,
 (6, 6): 2.4343534542967866}

In [68]:
#Set module attributes**** Need to change this depending on the condition

REWARD
nx.set_node_attributes(modg, reward_clust_dict, 'clustering')

NameError: name 'REWARD' is not defined

In [69]:
#list(modg.nodes)

### Function to make module structure graph

In [70]:
def module_fig(G):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #nodes, size = zip(*nx.get_node_attributes(G,'clustering').items())
    
    
    positions=nx.circular_layout(G)
    plt.figure(figsize=(25,20))
    
    color = np.array(list(G.nodes))
    nColormap=plt.cm.Spectral #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    cM=color.max()
    cm=color.min()
    nx.draw_networkx_nodes(G,positions, 
                           node_color=color, 
                           node_size=1000,
                           alpha=1.0, 
                           cmap= 'Spectral',
                           vmin=cm,vmax=cM )
   
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, font_family='sans-serif')
    
    x=nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid', width = weights, edge_color = weights)
   
    edge_bar=plt.colorbar(x)
    edge_bar.set_label('Strength of edge weight',fontsize = 25)
    
    plt.title("Module Connectivity Weights", fontsize = 30)
    #plt.savefig(os.path.join(basepath,"betaseries_bevel/5_analysis/results/modularity_edges_reward_weighted.png"), format="PNG")
    plt.axis('off')
    plt.show()

In [71]:
module_fig(modg)

### Calculate Nodal Metrics by Participant
- Clustering Coefficient
- Betweenness centrality

Function to pull the nodal parameters by subject

In [72]:
def make_node_stats(list_o_data, direction, min_cor):
    clustering_dict ={}
    centrality_dict ={}
    pc_dict ={}
    
    for key, values in list_o_data.items():
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            tG = create_corr_network_5(G, direction,min_cor)
            ########################################
            clustering = nx.clustering(tG, weight=True)
            ########################################
            clustering_dict.setdefault(key, []).append(clustering)
            ########################################
            centrality = nx.betweenness_centrality(tG, weight=True)
            ########################################
            centrality_dict.setdefault(key, []).append(centrality)
            ########################################
            cor_mat = np.array(cor_matrix)
            test_array=np.array(list(list(b)))
            testPART=np.vstack(test_array)
            PC=bct.participation_coef(W=cor_mat, ci= testPART)
            pc_dict.setdefault(key, []).append(PC)
            ########################################
    return({'clustering_coeff':clustering_dict, 'btn_centrality':centrality_dict, 'pc': pc_dict})

In [73]:
#Metrics computed on an UNTHRESHOLDED GRAPH
nodes = make_node_stats(cor_dict, "positive", 0)

In [74]:
nodes.keys()

dict_keys(['clustering_coeff', 'btn_centrality', 'pc'])

In [75]:
#nodes['pc']['sub-001_punish']

In [76]:
pc=[]

for key,value in nodes['pc'].items():
    pc.append(value[0])

In [77]:
df_pc = pd.DataFrame.from_dict(pc)

In [78]:
df_pc['module0_mean'] = df_pc[[0,1,6,7]].mean(axis=1)
df_pc['module1_mean'] = df_pc[[2,3,16,17,22,23,26,27]].mean(axis=1)
df_pc['module2_mean'] = df_pc[[4,14,15]].mean(axis=1)
df_pc['module3_mean'] = df_pc[[5,10,11,12,13]].mean(axis=1)
df_pc['module4_mean'] = df_pc[[8,9,24,25]].mean(axis=1)
df_pc['module5_mean'] = df_pc[[18,19]].mean(axis=1)
df_pc['module6_mean'] = df_pc[[20,21]].mean(axis=1)

In [79]:
df_pc.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,vlThalamus_R,vmPFC_L,vmPFC_R,module0_mean,module1_mean,module2_mean,module3_mean,module4_mean,module5_mean,module6_mean
0,0.571556,0.888788,0.954064,0.958756,0.942525,0.932004,0.918974,0.951196,0.943987,0.947075,...,0.946704,0.952699,0.865899,0.832628,0.927662,0.946066,0.941678,0.945833,0.903026,0.924353
1,0.953607,0.947449,0.946155,0.946221,0.928702,0.818516,0.947052,0.935631,0.946395,0.947546,...,0.957604,0.947317,0.941964,0.945935,0.942794,0.927715,0.387442,0.951448,0.951351,0.948752
2,0.832874,0.867329,0.933124,0.943880,0.923769,0.915855,0.852712,0.858737,0.738954,0.935204,...,0.871875,0.934292,0.922780,0.852913,0.930173,0.930772,0.917356,0.860428,-4.979634,0.912995
3,0.931202,0.908220,0.919500,0.875618,0.937099,0.832082,0.898763,0.834872,0.927159,0.906161,...,0.939235,0.922853,0.923105,0.893264,0.886676,0.935258,-1.484177,0.924892,-4.945798,0.788853
4,0.915209,0.944087,0.940360,0.956456,0.950098,0.946342,0.953112,0.954031,0.956992,0.933100,...,0.955584,0.953290,0.938274,0.941610,0.949711,0.951941,0.949488,0.949390,0.565721,0.774715
5,0.887943,0.804314,0.926021,0.934881,0.821707,0.764589,0.902097,0.872820,0.880083,0.930645,...,0.877380,0.875020,0.888771,0.866794,0.892295,0.609453,-2.335151,0.879307,-15.658806,0.902968
6,0.951924,0.927935,0.938179,0.934813,0.948772,0.942674,0.937014,0.940892,0.947765,0.929235,...,0.902554,0.953240,0.939466,0.939441,0.932281,0.946688,0.935804,0.928327,0.937660,0.916779
7,0.763802,0.938942,0.949491,0.953439,0.950767,0.946219,0.947319,0.936865,0.950578,0.951338,...,0.952823,0.947564,0.939637,0.896732,0.949422,0.952355,0.935811,0.949892,0.861672,0.943202
8,0.906373,0.945398,0.953283,0.951406,0.949808,0.753046,0.953184,0.932485,0.947603,0.944298,...,0.929575,0.946705,0.954427,0.934360,0.948214,0.839088,0.908739,0.934127,-2.236058,0.946342
9,0.685450,-0.417713,0.776744,0.768992,0.839462,0.774978,-3.549285,0.861005,0.778497,0.784074,...,0.931745,-0.604984,0.783586,-0.605136,0.342157,0.834426,0.737936,-0.104017,0.648071,0.815889


In [80]:
clustering_coeff=[]

for key,value in nodes['clustering_coeff'].items():
    clustering_coeff.append(value[0])

In [81]:
#bring into pandas
df_clust = pd.DataFrame.from_dict(clustering_coeff)

In [82]:
#Calculate average module centrality *** THIS IS THE MEAN ASSIGNMENT
#df_clust['module0_mean'] = df_clust[[0,1,6,7]].mean(axis=1)
#df_clust['module1_mean'] = df_clust[[2,3,16,17]].mean(axis=1)
#df_clust['module2_mean'] = df_clust[[4,14,15]].mean(axis=1)
#df_clust['module3_mean'] = df_clust[[5,10,11]].mean(axis=1)
#df_clust['module4_mean'] = df_clust[[8,9,24,25]].mean(axis=1)
#df_clust['module5_mean'] = df_clust[[12,13]].mean(axis=1)
#df_clust['module6_mean'] = df_clust[[18,19]].mean(axis=1)
#df_clust['module7_mean'] = df_clust[[20,21]].mean(axis=1)
#df_clust['module8_mean'] = df_clust[[22,23,26,27]].mean(axis=1)

In [83]:
#Calculate average module centrality *** THIS IS THE MEAN ASSIGNMENT
df_clust['module0_mean'] = df_clust[[0,1,6,7]].mean(axis=1)
df_clust['module1_mean'] = df_clust[[2,3,16,17,22,23,26,27]].mean(axis=1)
df_clust['module2_mean'] = df_clust[[4,14,15]].mean(axis=1)
df_clust['module3_mean'] = df_clust[[5,10,11,12,13]].mean(axis=1)
df_clust['module4_mean'] = df_clust[[8,9,24,25]].mean(axis=1)
df_clust['module5_mean'] = df_clust[[18,19]].mean(axis=1)
df_clust['module6_mean'] = df_clust[[20,21]].mean(axis=1)

In [84]:

df_clust.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,vlThalamus_R,vmPFC_L,vmPFC_R,module0_mean,module1_mean,module2_mean,module3_mean,module4_mean,module5_mean,module6_mean
0,0.888889,0.938095,0.877493,0.877493,0.906667,0.894928,0.889328,0.877493,0.920290,0.913333,...,0.913333,0.895385,0.929825,0.898451,0.894942,0.895299,0.904638,0.915072,0.927950,0.896293
1,0.868946,0.868946,0.978261,0.978261,0.956667,1.000000,0.868946,0.876923,0.934783,0.926667,...,0.868946,0.978261,0.956667,0.870940,0.973920,0.944942,0.959517,0.899835,0.933718,0.912806
2,0.935673,0.880952,0.851449,0.827692,0.833333,0.831169,0.935673,0.926316,0.866667,0.826087,...,0.868421,0.830000,0.849802,0.919653,0.840267,0.831744,0.841569,0.849168,0.857277,0.869659
3,0.800725,0.852381,0.852381,0.815789,0.742754,0.800000,0.816993,0.732026,0.826087,0.857895,...,0.763333,0.818182,0.842105,0.800531,0.832407,0.771884,0.786453,0.813410,0.811846,0.843452
4,0.940711,0.908832,0.966667,0.935385,0.938462,0.913846,0.908832,0.908832,0.908832,0.926667,...,0.908832,0.966667,0.976285,0.916802,0.949471,0.948376,0.931902,0.924416,0.976190,0.946190
5,0.766234,0.807018,0.857143,0.830040,0.816993,0.733333,0.830040,0.868421,0.780952,0.729231,...,0.815789,0.847953,0.836842,0.817928,0.841256,0.713072,0.760834,0.782474,0.663743,0.827019
6,0.929231,0.944664,0.929231,0.926667,0.917379,0.917379,0.942029,0.926154,0.917379,0.939394,...,0.928854,0.917379,0.926667,0.935519,0.930425,0.920304,0.929270,0.927385,0.923205,0.957237
7,0.985294,0.926154,0.950769,0.923077,0.926154,0.976667,0.947692,0.960000,0.950769,0.923077,...,0.947692,0.923077,0.956667,0.954785,0.940353,0.941966,0.953129,0.942308,0.966304,0.939253
8,0.904762,0.920000,0.920000,0.901538,0.920000,0.915033,0.883191,0.909420,0.984190,0.916667,...,0.916667,0.883191,0.920000,0.904343,0.914597,0.947711,0.927343,0.938076,0.914545,0.920000
9,1.000000,0.945455,1.000000,1.000000,0.627451,1.000000,0.846154,0.683333,1.000000,1.000000,...,0.615942,0.924242,1.000000,0.868735,0.930227,0.788671,0.917091,0.885046,0.941377,0.719079


In [85]:
central=[]

for key,value in nodes['btn_centrality'].items():
    central.append(value[0])

In [86]:
df_cent = pd.DataFrame.from_dict(central)

In [87]:
df_cent['module0_mean'] = df_cent[[0,1,6,7]].mean(axis=1)
df_cent['module1_mean'] = df_cent[[2,3,16,17,22,23,26,27]].mean(axis=1)
df_cent['module2_mean'] = df_cent[[4,14,15]].mean(axis=1)
df_cent['module3_mean'] = df_cent[[5,10,11,12,13]].mean(axis=1)
df_cent['module4_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
df_cent['module5_mean'] = df_cent[[18,19]].mean(axis=1)
df_cent['module6_mean'] = df_cent[[20,21]].mean(axis=1)

In [88]:
#Calculate average module centrality
#df_cent['module0_mean'] = df_cent[[0,1,6,7]].mean(axis=1)
#df_cent['module1_mean'] = df_cent[[2,3,16,17]].mean(axis=1)
#df_cent['module2_mean'] = df_cent[[4,14,15]].mean(axis=1)
#df_cent['module3_mean'] = df_cent[[5,10,11]].mean(axis=1)
#df_cent['module4_mean'] = df_cent[[8,9,24,25]].mean(axis=1)
#df_cent['module5_mean'] = df_cent[[12,13]].mean(axis=1)
#df_cent['module6_mean'] = df_cent[[18,19]].mean(axis=1)
#df_cent['module7_mean'] = df_cent[[20,21]].mean(axis=1)
#df_cent['module8_mean'] = df_cent[[22,23,26,27]].mean(axis=1)

In [89]:
#rename_columns
df_cent.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,vlThalamus_R,vmPFC_L,vmPFC_R,module0_mean,module1_mean,module2_mean,module3_mean,module4_mean,module5_mean,module6_mean
0,0.002601,0.001956,0.006942,0.006942,0.004463,0.004740,0.004642,0.006942,0.003367,0.003975,...,0.003935,0.005329,0.001786,0.004036,0.004963,0.005218,0.004386,0.003803,0.002693,0.004240
1,0.012081,0.012081,0.000829,0.000829,0.001998,0.000000,0.012081,0.010758,0.005125,0.005870,...,0.012081,0.000829,0.001998,0.011750,0.001110,0.003339,0.001458,0.008789,0.004315,0.007039
2,0.001662,0.004793,0.007496,0.010707,0.009438,0.008311,0.001662,0.002178,0.002516,0.009086,...,0.004790,0.010315,0.007098,0.002574,0.008952,0.009696,0.008015,0.005938,0.006538,0.005763
3,0.012167,0.006306,0.006693,0.009017,0.023590,0.006548,0.008869,0.014118,0.009176,0.005338,...,0.018167,0.010647,0.007823,0.010365,0.007485,0.017060,0.010927,0.011248,0.004951,0.010003
4,0.002710,0.005757,0.001368,0.003528,0.003281,0.005074,0.005757,0.005757,0.005757,0.004185,...,0.005757,0.001368,0.000823,0.004995,0.002557,0.002628,0.003846,0.004500,0.000898,0.001910
5,0.017246,0.010978,0.006680,0.009052,0.009115,0.010538,0.009052,0.005063,0.014561,0.027346,...,0.008905,0.007757,0.009030,0.010585,0.008330,0.009015,0.011371,0.015311,0.013110,0.010825
6,0.003120,0.001900,0.003128,0.003129,0.004059,0.004059,0.002153,0.003340,0.004059,0.001873,...,0.002490,0.004059,0.003108,0.002628,0.003074,0.003819,0.003053,0.002857,0.003389,0.001292
7,0.000237,0.004147,0.002429,0.004574,0.004142,0.000863,0.002691,0.001843,0.002429,0.004574,...,0.002691,0.004574,0.001997,0.002230,0.003245,0.003193,0.002280,0.003096,0.001251,0.003090
8,0.004320,0.005048,0.004183,0.006621,0.004183,0.002865,0.008204,0.005260,0.000509,0.004069,...,0.004895,0.008204,0.004183,0.005708,0.005259,0.002292,0.003586,0.003399,0.002567,0.004183
9,0.000000,0.001701,0.000000,0.000000,0.049912,0.000000,0.007574,0.029653,0.000000,0.000000,...,0.109444,0.003015,0.000000,0.009732,0.018288,0.023010,0.005978,0.028115,0.003472,0.030416


Save out to CSV files

In [90]:
df_pc.to_csv(os.path.join(basepath,'betaseries_bevel/5_analysis/node_pc_reward_weighted_median.csv'), header = True, index = None)

In [ ]:
df_clust.to_csv(os.path.join(basepath,'betaseries_bevel/5_analysis/node_clustering_reward_weighted_median.csv'), header = True, index = None)

In [ ]:
df_cent.to_csv(os.path.join(basepath,'betaseries_bevel/5_analysis/node_centrality_reward_weighted_median.csv'), header = True, index = None)

# Check Partition Stability in Random Split Half

In [92]:
from random import sample 

In [93]:
sub_sample = sample(file_list,43)

In [94]:
print(sub_sample)

['/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-081_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-026_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-015_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-010_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-072_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-002_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-069_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-044_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-068_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-089_reward.txt', '/Users/jennygilbert/Document

In [104]:
random.shuffle(file_list)
file_list1 = file_list[:42]
file_list2 = file_list[42:]

In [106]:
#Test Partition in mean from file list1
my_dict={}
for item in file_list1:
    name=item.split('/')[7].split('.')[0]
    print(name)
    my_dict.setdefault(name, []).append(item)
    

data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))

        
cor_dict={}
for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    cor_dict[key] = cor_matrix

sub-016_reward
sub-056_reward
sub-021_reward
sub-005_reward
sub-028_reward
sub-047_reward
sub-014_reward
sub-087_reward
sub-088_reward
sub-001_reward
sub-072_reward
sub-031_reward
sub-077_reward
sub-071_reward
sub-070_reward
sub-011_reward
sub-063_reward
sub-015_reward
sub-054_reward
sub-013_reward
sub-020_reward
sub-017_reward
sub-024_reward
sub-010_reward
sub-089_reward
sub-082_reward
sub-030_reward
sub-040_reward
sub-026_reward
sub-036_reward
sub-038_reward
sub-032_reward
sub-083_reward
sub-085_reward
sub-061_reward
sub-058_reward
sub-041_reward
sub-068_reward
sub-090_reward
sub-053_reward
sub-045_reward
sub-004_reward


# Brain graphs  
http://visbrain.org/index.html

In [ ]:
#positions = pd.read_csv(os.path.join(basepath,'betaseries_bevel/3_pull_timeseries/betaseries_rois.txt'), sep='\t')

In [ ]:
#positions=positions.drop('Region', axis=1)


In [ ]:
#positions_dict=positions.to_dict('index')

In [ ]:
#nx.set_node_attributes(meanG, positions_dict, 'pos')

In [ ]:
def brain_graph(G, strength, radius, colormap):
    mods=[node[1]['modules'] for node in G.nodes(data=True)]
    ccol=[]
    for m in mods:
        if m == 0.0:
            ccol.append('mediumvioletred')
        if m == 1.0:
            ccol.append('orange')
        if m == 2.0:
            ccol.append('yellowgreen')
        if m == 3.0:
            ccol.append('slateblue')
            
    # Colorbar default arguments. See `visbrain.objects.ColorbarObj`
    CBAR_STATE = dict(cbtxtsz=12, txtsz=10., width=.1, cbtxtsh=3., ndigits=1,
                  rect=(-.3, -2., 1., 4.))
        
    nodes, edges = np.asarray([list(node[1]['pos'].values()) for node in meanG.nodes(data=True)]), nx.to_numpy_array(G)

    #Change backgrounnd color here
    sc = SceneObj(size=(500, 600), bgcolor = 'grey')
    
    # Coloring method
    color_by = 'strength'
    # Because we don't want to plot every connections, we only keep connections
    select = edges > strength
    
    # Define the connectivity object
    c_default = ConnectObj('default', nodes, edges, select=select, line_width=3., cmap=colormap, color_by=color_by)
    
    # Then, we define the sources
    s_obj = SourceObj('sources', nodes, color=ccol, radius_min=radius)
    sc.add_to_subplot(c_default, title='Node Connectivity Grouped by Module')
    
    # And add connect, source and brain objects to the scene
    sc.add_to_subplot(s_obj)
    sc.add_to_subplot(BrainObj('B3'), use_this_cam=True)
    
    #Add colorbar
    cb_parr = ColorbarObj(c_default, cblabel='Connectivity strength', **CBAR_STATE, cmap=colormap)
    sc.add_to_subplot(cb_parr, row=0, col=1, width_max=200)
    
    # Coloring method
    color_by = 'count'
    
    # Define the connectivity and source object
    c_count = ConnectObj('default', nodes, edges, select=select, line_width=4.,
                         color_by=color_by, antialias=True,  cmap=colormap)

    s_obj_c = SourceObj('sources', nodes, color=ccol, radius_min=radius,
                        symbol='square')

    # And add connect, source and brain objects to the scene
    sc.add_to_subplot(c_count, row=1, col=0,
                      title='Node Grouped by Number of Connections per node')

    sc.add_to_subplot(s_obj_c, use_this_cam=True, row=1, col=0)
    sc.add_to_subplot(BrainObj('B3'), use_this_cam=True, row=1, col=0)
    
    
    #Add colorbar
    cb_parr_c = ColorbarObj(c_count, cblabel='Number of connections per node', **CBAR_STATE, cmap=colormap)
    sc.add_to_subplot(cb_parr_c, row=1, col=1, width_max=200)

    sc.preview()

    #sc.screenshot(os.path.join(basepath,'betaseries_bevel/5_analysis/example_brain_obj.png'), transparent=True)

In [ ]:
#brain_graph(meanG, .4, 15., 'Blues')

## Save GRAPHS dictionary in a pickle file in case of crash

In [ ]:
pickle.dump(GRAPHS, open(os.path.join(basepath, 'betaseries_bevel/tmp/Graphs'), 'wb'), protocol=4)

In [ ]:
with open(os.path.join(basepath, 'betaseries_bevel/tmp/Graphs'), 'rb') as pickle_file:
    try:
        while True:
            GRAPHS = pickle.load(pickle_file)
#             print (GRAPHS)
    except EOFError:
        pass